In [1]:
%load_ext autoreload
%autoreload 2
# %env CUDA_DEVICE_ORDER=PCI_BUS_ID
# %env CUDA_VISIBLE_DEVICES=1
# %env CUDA_LAUNCH_BLOCKING=1
%env MKL_NUM_THREADS=1
%env OMP_NUM_THREADS=1
%env NUMEXPR_NUM_THREADS=1
%env MKL_DEBUG_CPU_TYPE=5

env: MKL_NUM_THREADS=1
env: OMP_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: MKL_DEBUG_CPU_TYPE=5


In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2022-06-28 08:59:31.911476: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-06-28 08:59:31.936549: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2399990000 Hz
2022-06-28 08:59:31.939630: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa368000b60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-06-28 08:59:31.939690: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-06-28 08:59:31.943692: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/nvidia/hpc_sdk/Linux_x86_64/21.2/compilers/extras/qd/lib:/opt/nvidia/hpc_sdk/Linux_x86_64/21.2/comm_libs/11.2/nccl/lib:/opt/nvidia/hpc_sdk/Linux_

# Set up the gym and wrap a monitor around it that will periodically record movies as it learns

In [3]:
import gym
from clusgym_env_unique_minima  import MCSEnv
from callback_simple import Callback
from tensorforce.execution import Runner
import gym.wrappers 
import numpy as np
import tensorforce
import copy
import tensorflow as tf
import os

In [4]:
#

In [5]:
timesteps = 500
num_parallel = 32
seed = 30
eleNames = ['Ni']
eleNums = [8]
clus_seed = 100
save_dir = './result_multi_env/'

In [6]:
def setup_env(recording=True, structure=None, structure_idx=None):
    
    # Set up gym
    MCS_gym = MCSEnv(eleNames=eleNames,
                     eleNums=eleNums,
                     clus_seed=clus_seed,
                     observation_fingerprints=True, 
                     save_dir = save_dir,
                     timesteps = timesteps,
                     save_every = 1,
                    )
    
    if recording:
    # Wrap the gym to provide video rendering every 50 steps
        MCS_gym = gym.wrappers.Monitor(MCS_gym, 
                                         os.path.join(save_dir, 'vid'), 
                                         force=True,
                                        video_callable = lambda episode_id: (episode_id+1)%30==0) #every 50, starting at 51
    
    #Convert gym to tensorforce environment
    env = tensorforce.environments.OpenAIGym(MCS_gym,
                                         max_episode_timesteps=timesteps,
                                         visualize=False)
    
    return env
    
env = setup_env().environment.env
print('initial energy', env.initial_energy)

initial energy 11.252397013520229


/opt/conda/lib/python3.8/site-packages/gym/wrappers/monitor.py:86: UserWarning: WARN: Trying to monitor an environment which has no 'spec' set. This usually means you did not create it via 'gym.make', and is recommended only for advanced users.
  logger.warn(


In [7]:
print(env.atoms)
print(env.atoms.get_positions())
from ase.io import write
write("initial_clus.traj", env.atoms)

Atoms(symbols='Ni8', pbc=True, cell=[22.476408257419912, 24.757224523586167, 21.445956314611653], calculator=EMT(...))
[[ 9.14795472 13.29311352 11.21427005]
 [11.39683897 12.62050761 11.97393969]
 [10.72578025 14.9645181  11.96131213]
 [10.3375256  11.55059875  9.99042448]
 [ 9.49096169 11.12674832 12.2275232 ]
 [11.60442279 10.2688204  11.54775543]
 [12.92006015 14.34479348 11.28625192]
 [11.09164477 13.84491073  9.83883306]]


In [ ]:
#

# Set up the gym and agent in tensorforce

In [8]:
from tensorforce.agents import Agent
tf.random.set_seed(seed)
agent = Agent.create(
    agent=dict(type='trpo'),
#     agent=dict(type='trpo', critic_network='auto', critic_optimizer=1.0),
    environment=setup_env(), 
    batch_size=1,
    learning_rate=1e-3,
    memory = 50000,
#     memory = dict(type='replay',capacity=10000),
    max_episode_timesteps = timesteps,
    exploration=dict(
        type='decaying', unit='timesteps', decay='exponential',
        initial_value=0.2, decay_steps=50000, decay_rate=0.5 #10000, 1000000
    ),
    
    parallel_interactions = num_parallel,
    
)
    

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


# Run the DRL method in parallel (multiple environments)

In [ ]:
num_episodes = num_parallel*640
callback = Callback(save_dir).episode_finish

runner = Runner(
    agent=agent,
    environments=[setup_env(recording=False) for _ in range(num_parallel)],
    num_parallel=num_parallel,
    remote='multiprocessing',
    max_episode_timesteps=timesteps,
)

#runner.run(num_episodes=num_parallel*250, callback=callback, callback_episode_frequency=1)
runner.run(num_episodes=num_episodes, callback=callback, callback_episode_frequency=1)
# runner.run(num_episodes=100, evaluation=True)
runner.close()


Episodes:   2%|▏         | 428/20480 [03:10, reward=-14069.91, ts/ep=107, sec/ep=23.39, ms/ts=218.6, agent=1.4%, comm=84.6%]

# Run a single Env

In [9]:

#plot_frequency --> plotting energy and trajectories frequency
callback = Callback(save_dir).episode_finish

runner2 = Runner(
    agent=agent,
    environment=setup_env(recording=False),
    max_episode_timesteps=timesteps,
)

# %prun runner.run(num_episodes=2, callback=callback, callback_episode_frequency=1)

# callback_episode_frequency --> saving results and trajs frequency
runner2.run(num_episodes=100, callback=callback, callback_episode_frequency=1)
# runner2.run(num_episodes=100, evaluation=True)
# runner2.close()

Episodes: 100%|██████████| 100/100 [12:46, reward=-108675.00, ts/ep=423, sec/ep=14.70, ms/ts=34.8, agent=7.9%]